This notebook is for creating a compiled dataset to work with.

In [3]:
import pickle
import numpy as np
import pandas as pd

In [4]:
sdata = pickle.load(open('typing_sdata.pickle', 'rb'))
subject_list = list(sdata.keys())

In [14]:
subject_data = sdata[subject_list[0]]
typing_histograms = subject_data[0]
num_typing_sessions_for_subject = len(typing_histograms)

In [15]:
num_typing_sessions_for_subject

409

In [22]:
import pandas as pd
import os
import pickle
USER_DATA_ROOT = 'D:/Cornell Academics/Fall/Product Studio/iPrognosis Data/iprognosis_data/iprognosis_typing_dev/Data'

In [67]:
def condition_frame(df):
    df['Release'] = df['Release'].apply(lambda x: x.split(' ')[1])
    
    return(df)

In [70]:
def user_to_frame(user_id, path):
    files = os.listdir(path)
    frames = []
    cols = ['Junk', 'Press', 'Release', 'Normed Pressure']
    for fi in files:
        df = pd.read_csv(f'{path}/{fi}', names=cols)
        df = condition_frame(df)
        df['Subject ID'] = user_id  # Add the SubjectID column
        frames.append(df)
    big_frame = pd.concat(frames)
    return big_frame


In [71]:
def main():
    user_list = [f'S0{x}' for x in range(1, 10)]
    user_list.extend([f'S{x}' for x in range(10, 34)])
    frames = []
    for user_id in user_list:
        path = f'{USER_DATA_ROOT}/{user_id}'
        frame = user_to_frame(user_id, path)  # Call user_to_frame with the user_id
        frames.append(frame)
    full_data_frame = pd.concat(frames, ignore_index=True)  # Concatenate all frames and reset the index
    return full_data_frame

In [72]:
frames_master = main()

In [88]:
frames_master['Subject ID'] = frames_master['Subject ID'].str.lstrip('S').str.lstrip('0').astype(int)

In [89]:
frames_master.head()

,Junk,Press,Release,Normed Pressure,Subject ID
0,Press,16144290,16144380,0.5750,1
1,Press,16156363,16156478,0.3875,1
2,Press,16157517,16157625,0.5625,1
3,Press,16158215,16158330,0.5000,1
4,Press,16160974,16161072,0.6250,1


In [91]:
patient_info_df = pd.read_csv('iprognosis_typing_dev/Demographics_Clinical_Characteristics.csv')

In [92]:
frames_master['Subject ID'].dtypes

dtype('int32')

In [139]:
merged_df = frames_master.merge(patient_info_df, on='Subject ID', how='left')

In [140]:
merged_df.columns

Index(['Junk', 'Press', 'Release', 'Normed Pressure', 'Subject ID',
       'Age (years)', 'Gender', 'Education level', 'Years of smartphone usage',
       'Dominant hand', 'Group', 'Years from diagnosis', 'Hoehn & Yahr stage',
       'Most affected side (PD patients only)',
       'Levodopa Equivalent Daily Dose (LEDD) (mg)', 'UPDRS_III Total score',
       'UPDRS_III Item 21 Tremor-Right hand',
       'UPDRS_III Item 21 Tremor-Left hand',
       'UPDRS_III Item 22 Rigidity-Right hand',
       'UPDRS_III Item 22 Rigidity-Left hand',
       'UPDRS_III Item 23 Finger Taps-Right hand',
       'UPDRS_III Item 23 Finger Taps-Left hand',
       'UPDRS_III Item 31 Body Bradykinesia/Hypokinesia'],
      dtype='object')

In [141]:
columns_to_drop = ['Junk','Dominant hand','Hoehn & Yahr stage',
       'Most affected side (PD patients only)','Years from diagnosis',
       'Levodopa Equivalent Daily Dose (LEDD) (mg)', 'UPDRS_III Total score',
       'UPDRS_III Item 21 Tremor-Right hand',
       'UPDRS_III Item 21 Tremor-Left hand',
       'UPDRS_III Item 22 Rigidity-Right hand',
       'UPDRS_III Item 22 Rigidity-Left hand',
       'UPDRS_III Item 23 Finger Taps-Right hand',
       'UPDRS_III Item 23 Finger Taps-Left hand',
       'UPDRS_III Item 31 Body Bradykinesia/Hypokinesia']
merged_df =merged_df.drop(columns=columns_to_drop)

In [142]:
merged_df['Release'] = merged_df['Release'].astype('int32')

In [143]:
merged_df['delta'] = merged_df['Release'].subtract(merged_df['Press'])

In [144]:
merged_df['Wait_Time'] = merged_df.groupby('Subject ID').apply(
    lambda group: group['Press'] - group['Release'].shift(1)
).reset_index(level=0, drop=True)


In [147]:
merged_df

,Press,Release,Normed Pressure,Subject ID,Age (years),Gender,Education level,Years of smartphone usage,Group,delta,Wait_Time
0,16144290,16144380,0.5750,1,69,Male,High school graduate,3,PD,90,NaN
1,16156363,16156478,0.3875,1,69,Male,High school graduate,3,PD,115,11983.0
2,16157517,16157625,0.5625,1,69,Male,High school graduate,3,PD,108,1039.0
3,16158215,16158330,0.5000,1,69,Male,High school graduate,3,PD,115,590.0
4,16160974,16161072,0.6250,1,69,Male,High school graduate,3,PD,98,2644.0
...,...,...,...,...,...,...,...,...,...,...,...
26533,610585103,610585160,0.7250,33,53,Male,University graduate,6,Control,57,176.0
26534,610585427,610585484,0.6000,33,53,Male,University graduate,6,Control,57,267.0
26535,610585943,610585992,0.8375,33,53,Male,University graduate,6,Control,49,459.0
26536,610589510,610589573,0.7375,33,53,Male,University graduate,6,Control,63,3518.0


In [149]:
merged_df.to_csv('Patient_Keyboard_Data_Final.csv')